# Company Brochure Generation

In [1]:
# Author: Sushant Mittal
# LinkedIn: https://www.linkedin.com/in/sushantmittal-sm/
print("Hello :)")

Hello :)


In [2]:
# Loading the libraries:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

import sys
sys.path.append('../')
from scraper import fetch_website_contents, fetch_website_links

In [3]:
# Connecting with Ollama and testing Connection:
load_dotenv()
ollama = OpenAI(
    api_key="OLLAMA",
    base_url="http://localhost:11434/v1"
)

openr = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

MODEL="qwen2.5"
response = ollama.chat.completions.create(
    model=MODEL,
    messages=[
        {"role":"system", "content":"You are a helpful assistant."},
        {"role":"user", "content":"Which model are you?"},
    ]
)

response.choices[0].message.content

"I'm a large language model created by Alibaba Cloud, and I go by the name Qwen."

In [4]:
# Getting the links for the website that are appropriate for Brochure:

link_system_prompt = """
You are a expert in selecting relevant information required to be published in the Brochure. Provided the URLs from the website you pick the ones that contains most relevant information for Brochure creation.
You provide the output of selected URLs along with assigned name in JSON format as following:
[
    {"name": "about page", "url": "https://full.url/goes/here/about"},
    {"name": "careers page", "url": "https://another.full.url/careers"}
]
"""

def get_link_user_prompt(url):
    user_prompt = f"""
    Here are the links on the website {url}:
    Kindly think and decide which of them can contain relevant Brochure information. Reply the relevant ones with full URLs in JSON Format. 
    Don't include links that contain private information, emails and contact information. The output should strictly return a JSON in expected format with no additional information, so that it can be converted to python dictionary with json.loads().
    
    Links:
    """
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt


def get_relevant_links_ollama(url):
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system", "content":link_system_prompt},
            {"role":"user", "content":get_link_user_prompt(url)},
        ],
        response_format={"type":"json_schema"}
    )
    return response.choices[0].message.content

def get_relevant_links_openr(url):
    response = openr.chat.completions.create(
        model="nvidia/nemotron-3-nano-30b-a3b:free",
        messages=[
            {"role":"system", "content":link_system_prompt},
            {"role":"user", "content":get_link_user_prompt(url)},
        ],
        #response_format={"type":"json_schema"}
    )
    return response.choices[0].message.content

In [5]:
# url=input("Enter URL:")
# print(get_relevant_links_openr(url))

In [6]:
# Brochure Generation Code

brochure_system_prompt = """
You are an expert in designing Colorful and Markdown format Brochures for the company given the relevant content from various links/urls from the companies website.
"""

def get_brochure_user_prompt(url):
    brochure_user_prompt = """
    Here's the content from the website for various pages. Design the company Brochure using this information:

    Content:
    """
    webpage_contents = ["## "+link['name']+":\n"+fetch_website_contents(link['url'])+"\n\n" for link in json.loads(get_relevant_links_openr(url))]
    brochure_user_prompt += "\n".join(webpage_contents)
    return brochure_user_prompt

def get_brochure(url):
    response = openr.chat.completions.create(
        model="nvidia/nemotron-3-nano-30b-a3b:free",
        messages=[
            {"role":"system", "content":brochure_system_prompt},
            {"role":"user", "content":get_brochure_user_prompt(url)},
        ],
        #response_format={"type":"json_schema"}
    )
    return response.choices[0].message.content

In [7]:
brochure = get_brochure("https://www.huggingface.co")
display(Markdown(brochure))

# 🎨 **Hugging Face Enterprise Brochure**  
*Scale AI‑innovation with the world’s leading ML platform*  

---  

## 🚀 Why Choose Hugging Face Enterprise?  

| ✅ **Enterprise‑grade security** | 🛡️ SSO, SAML, granular RBAC, audit logs & token controls |
| ✅ **Global data residency** | 🌍 Choose & manage data‑center regions for every repo |
| ✅ **Full‑stack monitoring** | 📊 Real‑time analytics & usage dashboards |
| ✅ **Unlimited compute** | ⚡ ZeroGPU boost (5× quota) + advanced inference providers |
| ✅ **Dedicated support** | 🚑 Priority SLAs, legal/compliance assistance, onboarding experts |
| ✅ **Flexible finance** | 💰 Managed billing, annual commitments & custom contracts |
| ✅ **Private collaboration** | 📂 Private datasets, storage (1 TB / member + $25/TB), private storage |

> **“Everything you already love about Hugging Face, now in Enterprise mode.”**  

---  

## 🎯 Core Enterprise Features  

| Feature | What it does | Benefit |
| ------- | ------------ | ------- |
| **Single Sign‑On (SSO)** | Connect to your IdP (Okta, Azure AD, etc.) | Seamless, secure access for all users |
| **Region Management** | Host repos in US, EU, Asia, etc. | Keep data where compliance demands |
| **Audit Logs** | Full‑fidelity event logging | Traceability & governance |
| **Resource Groups** | Granular repo permissions | Fine‑tuned access control |
| **Token Management** | Centralised API‑token policies | Reduce leakage, enforce approvals |
| **Analytics Dashboard** | Repository usage, inference spend, storage | Data‑driven budgeting |
| **ZeroGPU Boost** | +5× quota for every org member | Faster experimentation |
| **Private Datasets Viewer** | Browse & collaborate on private data | No extra tooling required |
| **Private Storage** | 1 TB per member + $25/TB extra | Secure, centralized data lake |
| **Inference Providers** | Bill & monitor external inference services | Control costs, scale on demand |
| **Advanced Security** | Org‑wide security policies & default repo visibility | Harden the platform |
| **Billing & Commitments** | Yearly caps, managed invoicing | Predictable spend |
| **Priority Support** | 24/7 dedicated CSM | Faster issue resolution |
| **Custom Contracts** | Negotiated terms, enterprise SLAs | Tailored to your org |

---  

## 💼 Pricing Overview  

> **All prices are per user / month (USD)**  

| Plan | Price | Who it’s for | Key Inclusions |
|------|------|--------------|----------------|
| **PRO** | **$9** | Individuals & freelancers | 10× private storage, 20× inference credits, 8× ZeroGPU quota, PRO badge |
| **TEAM** | **$20 / user** | Growing teams | SSO/SAML, storage‑region control, audit logs, resource groups, analytics, priority support, ZeroGPU & inference benefits for all members |
| **ENTERPRISE** | **From $50 / user** | Large organisations | Everything in Team **+** Unlimited storage & bandwidth, managed annual billing, legal & compliance tooling, personalised onboarding, dedicated support, custom contracts |

> **Free Core Hub** – Open‑source models, datasets & Spaces remain **FREE** for public use.  

---  

## 🌈 Brand Assets (Use‑Ready)  

- **Logos** – `.svg`, `.png`, `.ai` (download from [Brand Assets](https://huggingface.co/brand-assets))  
- **Primary Colors**  
  - `#FFD21E` (Sunshine Yellow)  
  - `#FF9D00` (Electric Orange)  
  - `#6B7280` (Slate Gray)  
- **Brand Bio** – *Hugging Face is the collaboration platform for the machine‑learning community, empowering the next generation of AI engineers, scientists, and end‑users to learn, share, and build an open & ethical AI future.*  

> **Tip:** Use the yellow (#FFD21E) for headings, orange (#FF9D00) for call‑to‑action buttons, and slate gray for body text to keep a vibrant, on‑brand look.  

---  

## 📚 Documentation & Resources  

- **Hub & Client Libraries** – Python, JavaScript/TypeScript, and more  
- **Model & Dataset Hubs** – Search, filter, and discover > 1 M models & datasets  
- **Deployment & Inference** – Inference Endpoints, AWS/Azure/GCP integrations, TGI, TEI, and more  
- **Training & Optimization** – PEFT, Accelerate, Optimum, TRL, and specialized kernels  
- **Full Docs** – <https://huggingface.co/docs>  

> **Explore**: <https://huggingface.co/docs> (All libraries, tutorials, and API references)  

---  

## 📜 Quick Legal & Policy Highlights  

| Document | Effective Date | Key Takeaway |
|----------|----------------|--------------|
| **Terms of Service** | 15 Sept 2022 | Binding agreement; acceptance via continued use; right to modify with 10‑day notice |
| **Privacy Policy** | 28 Mar 2023 | Collects personal info for service improvement; 10‑day notice for policy updates |
| **Content & Code of Conduct** | Ongoing | Community standards & acceptable use rules |

> **Need legal help?** Contact **legal@huggingface.co**  

---  

## 📞 Get Started  

| Action | Button | Details |
|--------|--------|---------|
| **Start a Free Team Trial** | 👉 [Start Team](https://huggingface.co/team) | 14‑day full access, no credit card required |
| **Request Enterprise Quote** | 👉 [Contact Sales](https://huggingface.co/contact) | Custom pricing, flexible contracts |
| **Explore PRO Benefits** | 👉 [Upgrade to PRO](https://huggingface.co/pro) | Boost personal workflow today |
| **Download Brand Kit** | 👉 [Brand Assets](https://huggingface.co/brand-assets) | Logos, color palettes, usage guide |

---  

### 🎉 **Join the AI Revolution**  

> **Hugging Face Enterprise** gives you the power to build, ship, and scale AI—securely, reliably, and with enterprise‑grade governance.  

**Ready to supercharge your organization?**  

> **[Contact Sales]** | **[Start Free Team]** | **[Upgrade to PRO]**  

---  

*Prepared with the official Hugging Face brand assets and documentation (April 2025). All information reflects the latest public pricing and feature set.*

In [8]:
# Brochure Translation (English to Spanish)
translation_system_prompt = """
You're a translator that translates the given information in English to Spanish, maintaining the input format. For example: If Markdown format data is provided, you return the translated data in same Mardown format.
"""

def get_translation_user_prompt(brochure):
    translation_user_prompt = """
    Translate the following Brochure into Spanish:

    """
    translation_user_prompt += brochure
    return translation_user_prompt

def get_spanish_brochure(brochure):
    response = openr.chat.completions.create(
        model="nvidia/nemotron-3-nano-30b-a3b:free",
        messages=[
            {"role":"system", "content":translation_system_prompt},
            {"role":"user", "content":get_translation_user_prompt(brochure)},
        ],
        #response_format={"type":"json_schema"}
    )
    return response.choices[0].message.content


In [9]:
spanish_brochure = get_spanish_brochure(brochure)
display(Markdown(spanish_brochure))

# 🎨 **Brochure de Hugging Face Enterprise**  
*Escala la innovación AI con la plataforma ML líder del mundo*  

---  

## 🚀 ¿Por qué elegir Hugging Face Enterprise?  

| ✅ **Seguridad empresarial** | 🛡️ SSO, SAML, RBAC granular, registros de auditoría y controles de token |
| ✅ **Residencia global de datos** | 🌍 Elige y gestiona regiones de data‑center para cada repositorio |
| ✅ **Monitoreo de pila completa** | 📊 Analíticas en tiempo real y paneles de uso |
| ✅ **Computación ilimitada** | ⚡ ZeroGPU *boost* (5× cuota) + proveedores avanzados de inferencia |
| ✅ **Soporte dedicado** | 🚑 SLA de prioridad, asistencia legal/compliance, expertos de incorporación |
| ✅ **Finanzas flexibles** | 💰 Facturación gestionada, compromisos anuales y contratos personalizados |
| ✅ **Colaboración privada** | 📂 Conjuntos de datos y almacenamiento privados (1 TB / miembro + $25/TB) |

> **“Todo lo que ya amas de Hugging Face, ahora en modo Enterprise.”**  

---  

## 🎯 Funcionalidades clave para Enterprise  

| Característica | Qué hace | Beneficio |
| -------------- | -------- | --------- |
| **Single Sign‑On (SSO)** | Conecta con tu IdP (Okta, Azure AD, etc.) | Acceso sin interrupciones y seguro para todos los usuarios |
| **Gestión de regiones** | Hospeda repos en US, EU, Asia, etc. | Mantén los datos donde exija la normativa |
| **Registros de auditoría** | Registro de eventos de alta fidelidad | Rastreabilidad y gobernanza |
| **Grupos de recursos** | Permisos granulares por repositorio | Control de acceso afinado |
| **Gestión de tokens** | Políticas centralizadas de tokens API | Reduce fugas, impone aprobaciones |
| **Panel de analíticas** | Uso de repos, gasto en inferencia, almacenamiento | Presupuesto basado en datos |
| **ZeroGPU Boost** | +5× cuota para cada miembro de la org. | Experimentos más rápidos |
| **Visor de conjuntos de datos privados** | Navega y colabora en datos privados | Sin herramientas adicionales |
| **Almacenamiento privado** | 1 TB por miembro + $25/TB extra | Lago de datos seguro y centralizado |
| **Proveedores de inferencia** | Factura y monitoriza servicios externos | Controla costos, escala bajo demanda |
| **Seguridad avanzada** | Políticas de seguridad a nivel de org. y visibilidad predeterminada de repos | Endurecimiento de la plataforma |
| **Facturación y compromisos** | Límite anual gestionado, facturación simplificada | Gasto predecible |
| **Soporte prioritario** | CSM dedicado 24/7 | Resolución de incidencias más rápida |
| **Contratos personalizados** | Términos negociados, SLA empresariales a medida | Adaptado a tu organización |

---  

## 💼 Resumen de precios  

> **Todos los precios son por usuario / mes (USD)**  

| Plan | Precio | A quién va dirigido | Principales inclusiones |
|------|--------|--------------------|--------------------------|
| **PRO** | **$9** | Individuos y freelancers | 10× almacenamiento privado, 20× créditos de inferencia, 8× cuota ZeroGPU, insignia PRO |
| **TEAM** | **$20 / usuario** | Equipos en crecimiento | SSO/SAML, control de regiones de almacenamiento, registros de auditoría, grupos de recursos, analíticas, soporte prioritario, ZeroGPU e inferencia para todos los miembros |
| **ENTERPRISE** | **Desde $50 / usuario** | Grandes organizaciones | Todo lo de **TEAM** **+** Almacenamiento y ancho de banda ilimitados, facturación anual gestionada, herramientas legales y de compliance, incorporación personalizada, soporte dedicado, contratos a medida |

> **Core Hub gratuito** – Modelos, conjuntos de datos y Spaces de código abierto siguen siendo **GRATIS** para uso público.  

---  

## 🌈 Activos de marca (Listos para usar)  

- **Logotipos** – `.svg`, `.png`, `.ai` (descárgalos en [Brand Assets](https://huggingface.co/brand-assets))  
- **Colores primarios**  
  - `#FFD21E` (Amarillo Sol)  
  - `#FF9D00` (Naranja Eléctrico)  
  - `#6B7280` (Gris Pizarra)  
- **Bio de marca** – *Hugging Face es la plataforma de colaboración para la comunidad de aprendizaje automático, que empodera a la próxima generación de ingenieros, científicos y usuarios de IA para aprender, compartir y construir un futuro de IA abierto y ético.*  

> **Consejo:** Usa el amarillo (#FFD21E) para encabezados, el naranja (#FF9D00) para botones de llamada a la acción y el gris pizarra para el cuerpo del texto para mantener una apariencia vibrante y alineada con la marca.  

---  

## 📚 Documentación y recursos  

- **Hub y bibliotecas cliente** – Python, JavaScript/TypeScript y más  
- **Hubs de modelos y conjuntos de datos** – Busca, filtra y descubre > 1 M de modelos y conjuntos de datos  
- **Despliegue e inferencia** – Endpoints de inferencia, integraciones con AWS/Azure/GCP, TGI, TEI y más  
- **Entrenamiento y optimización** – PEFT, Accelerate, Optimum, TRL y kernels especializados  
- **Documentación completa** – <https://huggingface.co/docs>  

> **Explora:** <https://huggingface.co/docs> (Todas las bibliotecas, tutoriales y referencias de la API)  

---  

## 📜 Aspectos rápidos de derecho y políticas  

| Documento | Fecha de vigencia | Principio clave |
| --------- | ----------------- |-----------------|
| **Términos de Servicio** | 15 sep 2022 | Acuerdo vinculante; aceptación mediante uso continuo; derecho a modificar con 10 días de aviso |
| **Política de privacidad** | 28 mar 2023 | Recopila información personal para mejorar el servicio; 10 días de aviso para actualizaciones |
| **Contenido y Código de Conducta** | En vigor | Normas comunitarias y reglas de uso aceptable |

> **¿Necesitas ayuda legal?** Contacta a **legal@huggingface.co**  

---  

## 📞 Cómo comenzar  

| Acción | Botón | Detalles |
|--------|-------|----------|
| **Prueba gratuita del Equipo** | 👉 [Comienza Equipo](https://huggingface.co/team) | 14 días de acceso completo, sin tarjeta de crédito |
| **Solicita cotización Enterprise** | 👉 [Contacto Ventas](https://huggingface.co/contact) | Precios personalizados, contratos flexibles |
| **Descubre beneficios PRO** | 👉 [Actualiza a PRO](https://huggingface.co/pro) | Impulsa tu flujo de trabajo hoy |
| **Descarga el kit de marca** | 👉 [Activos de marca](https://huggingface.co/brand-assets) | Logotipos, paletas de color y guía de uso |

---  

### 🎉 **Únete a la Revolución de la IA**  

> **Hugging Face Enterprise** te brinda el poder de construir, lanzar y escalar IA – seguro, fiable y con gobernanza empresarial.  

**¿Listo para potenciar tu organización?**  

> **[Contacto Ventas]** | **[Prueba Gratuita del Equipo]** | **[Actualiza a PRO]**  

---  

*Preparado con los activos de marca y la documentación oficial de Hugging Face (abril 2025). Toda la información refleja el conjunto de precios y funcionalidades más reciente.*